In [1]:
import time
import pywikibot
import mwparserfromhell
import csv
import json
import re
import xml.sax
from bs4 import BeautifulSoup
import requests

In [3]:
site = pywikibot.Site("id", "wikipedia")
art = pywikibot.Page(site, 'Perang Diponegoro')


wikicode = mwparserfromhell.parse(art.get())
matches = wikicode.filter_templates(matches = "Infobox")
properties = {param.name.strip(): param.value
                                for param in matches[0].params}
print(properties)

{'conflict': 'Perang Diponegoro \n', 'image': '[[Berkas:Raden Saleh - Diponegoro arrest.jpg|300px]]\n', 'caption': 'Lukisan Peristiwa Penangkapan Pangeran Diponegoro oleh Raden Saleh \n', 'place': '[[Yogyakarta]], [[Surakarta]], [[Jawa Tengah]], dan [[Jawa Timur]]\n', 'date': '21 Juli [[1825]] - 9 Februari [[1830 1840]]\n', 'casus': 'Jalan yang dibangun Belanda melintasi makam leluhur Pangeran Diponegoro 6\n', 'result': 'Kemenangan Belanda\n*Penangkapan dan pengasingan Pangeran [[Diponegoro]] ke [[Makassar]]<ref name="sulawesi">Toby Alice Volkman: \'\'Sulawesi: island crossroads of Indonesia\'\', Passport Books, 1990, ISBN 0844299065,  [http://books.google.com/books?ei=ACmtT6-3C5LN4QST_72dDA&hl=de&id=ZNdwAAAAMAAJ&dq=java+war+Diponegoro+deported+makassar&q=%22deported+to+manado+and+then+to+makassar%22 page 73].</ref>\n', 'combatant1': '{{ubl|[[Imperium Belanda|Belanda]]|[[Kesultanan Yogyakarta]]|[[Pasukan Tulungan]]}}\n', 'combatant2': '{{ubl|Pasukan Jawa/Diponegoro}}\n', 'commander1': 

In [5]:
for key,value in properties.items():
    print(key,":",value)
    print("===========")

conflict : Perang Diponegoro 

image : [[Berkas:Raden Saleh - Diponegoro arrest.jpg|300px]]

caption : Lukisan Peristiwa Penangkapan Pangeran Diponegoro oleh Raden Saleh 

place : [[Yogyakarta]], [[Surakarta]], [[Jawa Tengah]], dan [[Jawa Timur]]

date : 21 Juli [[1825]] - 9 Februari [[1830 1840]]

casus : Jalan yang dibangun Belanda melintasi makam leluhur Pangeran Diponegoro 6

result : Kemenangan Belanda
*Penangkapan dan pengasingan Pangeran [[Diponegoro]] ke [[Makassar]]<ref name="sulawesi">Toby Alice Volkman: ''Sulawesi: island crossroads of Indonesia'', Passport Books, 1990, ISBN 0844299065,  [http://books.google.com/books?ei=ACmtT6-3C5LN4QST_72dDA&hl=de&id=ZNdwAAAAMAAJ&dq=java+war+Diponegoro+deported+makassar&q=%22deported+to+manado+and+then+to+makassar%22 page 73].</ref>

combatant1 : {{ubl|[[Imperium Belanda|Belanda]]|[[Kesultanan Yogyakarta]]|[[Pasukan Tulungan]]}}

combatant2 : {{ubl|Pasukan Jawa/Diponegoro}}

commander1 : {{ubl|[[Hendrik Merkus de Kock|Hendrik M. de Kock]]|

# military conflict

In [40]:
class Infobox():
    def __init__(self,title):
        self.title = title
        site = pywikibot.Site("id", "wikipedia")
        art = pywikibot.Page(site, self.title)
    

        wikicode = mwparserfromhell.parse(art.get())
        matches = wikicode.filter_templates(matches = "Infobox")
        self.infobox = {param.name.strip(): param.value
                            for param in matches[0].params}
        print(self.infobox)
        self.clean_infobox = self.clean_data()
    def clean_link(self,key):
        return ",".join(self.filter_wikilinks(self.infobox.get(key,"")))
    def clean_text(self,key):
        return self.text(self.infobox.get(key,""))
    def clean_list(self,key):
        return ",".join(self.list(self.infobox.get(key,"")))
    def filter_wikilinks(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        link = wikicode.filter_wikilinks()
        temp = []
        for i in link:
            wiki = mwparserfromhell.parse(i)
            temp.append(wiki.title())
        return temp
    def text(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        return re.sub("\n","",wikicode.strip_code().strip())
    def list(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        template = wikicode.filter_templates(matches="ubl")
        temp = []
        for i in template[0].params :
            temp.append(self.text(i))
        return temp

In [41]:
class Infobox_military_conflict(Infobox):
    def __init__(self,title):
        Infobox.__init__(self,title)
    def clean_data(self):
        temp = {}
        temp['conflict'] = self.clean_text('conflict')
        temp['image'] = self.clean_text('image')
        temp['caption'] = self.clean_text('caption')
        temp['date'] = self.clean_text('date')
        temp['place'] = self.clean_link('place')
        temp['casus'] = self.clean_text('casus')
        temp['teritory'] = self.clean_text('teritory')
        temp['result'] = self.clean_text('result')
        temp['combatant1'] = self.clean_list("combatant1")
        temp['combatant2'] = self.clean_list("combatant2")
        temp['commander1'] = self.clean_list("commander1")
        temp['commander2'] = self.clean_list("commander2")
        temp['strength1'] = self.clean_text('strength1')
        temp['strength2'] = self.clean_text('strength1')
        temp['casualties1'] = self.clean_text('casualties1')
        temp['casualties2'] = self.clean_text('casualties2')
        temp['notes'] = self.clean_text('notes')
        return temp
    

In [42]:
diponegoro = Infobox_military_conflict("Perang Diponegoro")
print(diponegoro.clean_infobox)

{'conflict': 'Perang Diponegoro \n', 'image': '[[Berkas:Raden Saleh - Diponegoro arrest.jpg|300px]]\n', 'caption': 'Lukisan Peristiwa Penangkapan Pangeran Diponegoro oleh Raden Saleh \n', 'place': '[[Yogyakarta]], [[Surakarta]], [[Jawa Tengah]], dan [[Jawa Timur]]\n', 'date': '21 Juli [[1825]] - 9 Februari [[1830 1840]]\n', 'casus': 'Jalan yang dibangun Belanda melintasi makam leluhur Pangeran Diponegoro 6\n', 'result': 'Kemenangan Belanda\n*Penangkapan dan pengasingan Pangeran [[Diponegoro]] ke [[Makassar]]<ref name="sulawesi">Toby Alice Volkman: \'\'Sulawesi: island crossroads of Indonesia\'\', Passport Books, 1990, ISBN 0844299065,  [http://books.google.com/books?ei=ACmtT6-3C5LN4QST_72dDA&hl=de&id=ZNdwAAAAMAAJ&dq=java+war+Diponegoro+deported+makassar&q=%22deported+to+manado+and+then+to+makassar%22 page 73].</ref>\n', 'combatant1': '{{ubl|[[Imperium Belanda|Belanda]]|[[Kesultanan Yogyakarta]]|[[Pasukan Tulungan]]}}\n', 'combatant2': '{{ubl|Pasukan Jawa/Diponegoro}}\n', 'commander1': 